In [4]:
import pandas as pd

xls = pd.ExcelFile('Planilhao_TCT_SPAECE_EF_2023_MT_240402-2 (1).xlsx')

df = pd.DataFrame(xls.parse('ESCOLA'))

df.columns = df.iloc[0]
df = df[1:]

# Remoção de Colunas 'CD_REDE','CD_ETAPA', 'CD_REGIONAL','CD_MUNICIPIO' 'CD_ESCOLA', manter apenas colunas com informações relevantes para análise de desempenho
df = df.drop(['CD_REDE','CD_ETAPA', 'CD_REGIONAL','CD_MUNICIPIO', 'CD_ESCOLA'], axis=1)

# Remover colunas completamente vazias (se existirem)
df = df.dropna(axis=1, how='all')

# Filtros para manter apenas os dados do 9º ano
df_9ano = df[df['DC_ETAPA'] == 'ENSINO FUNDAMENTAL DE 9 ANOS - 9º ANO'].reset_index(drop=True)

In [10]:
# Função para selecionar escola
def selecionar_escola(nome_escola):
    filtro = df['NM_ESCOLA'].str.contains(nome_escola, case=False, na=False)
    df_filtrado = df[filtro].copy()
    return df_filtrado

escola_selecionada = selecionar_escola("TEREZINHA FERREIRA PARENTE")

In [11]:
colunas_descritores = [col for col in df.columns if isinstance(col, str) and col.startswith("D")]
print(colunas_descritores)

['DC_REDE', 'DC_ETAPA', 'D01', 'D01_5EF', 'D02', 'D03', 'D04', 'D05', 'D05_SAEB', 'D06', 'D06_5EF', 'D06_SAEB', 'D07', 'D07_SAEB', 'D08', 'D09', 'D09_SAEB', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D17', 'D18', 'D19', 'D21', 'D24', 'D25', 'D26', 'D27', 'D30_SAEB', 'D32_SAEB', 'D35_SAEB', 'D37_SAEB', 'D45', 'D45_5EF', 'D46', 'D47', 'D48', 'D49', 'D50', 'D51', 'D52', 'D59', 'D59_5EF', 'D60', 'D61', 'D62', 'D63', 'D65', 'D66', 'D67', 'D69', 'D73', 'D74', 'D75', 'D77']


In [13]:
# Garantir que as colunas estão numéricas no top10_geral
top10_geral_todos = df_9ano.sort_values(by="VL_D", ascending=False).head(10)

# Substituir traços por NaN
top10_geral_todos[colunas_descritores] = top10_geral_todos[colunas_descritores].replace("-", pd.NA)
for col in colunas_descritores:
    top10_geral_todos[col] = pd.to_numeric(top10_geral_todos[col], errors='coerce')

C:\Users\TiagoVettorazzi\AppData\Local\Temp\ipykernel_23412\2332836250.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  top10_geral_todos[colunas_descritores] = top10_geral_todos[colunas_descritores].replace("-", pd.NA)


In [14]:
# Reconstruir o DataFrame comparativo_completo com todos os descritores (com sufixos)
desempenho_tfp_completo = escola_selecionada[colunas_descritores].T
desempenho_tfp_completo.columns = ["TFP - (%)"]
desempenho_tfp_completo = desempenho_tfp_completo.apply(pd.to_numeric, errors='coerce')
desempenho_tfp_completo.dropna(inplace=True)

# Garantir a existência da média dos 10 melhores com sufixo
media_top10_completo = top10_geral_todos[colunas_descritores].mean().dropna().to_frame(name="Top 10 Geral - Média (%)")

# Recriar o comparativo
comparativo_completo = desempenho_tfp_completo.join(media_top10_completo, how='inner')



In [16]:
# Definição de eixos
eixos_classificados = {
    'Números e Operações': [
        'D01_5EF', 'D06_5EF', 'D07', 'D08', 'D10', 'D11', 'D12', 'D13',
        'D15', 'D17', 'D18', 'D19', 'D21', 'D24', 'D25', 'D26', 'D27',
        'D30_SAEB', 'D32_SAEB', 'D35_SAEB'
    ],
    'Espaço e Forma': [
        'D45_5EF', 'D48', 'D49', 'D50', 'D51', 'D52', 'D06_SAEB', 'D07_SAEB'
    ],
    'Grandezas e Medidas': [
        'D59_5EF', 'D65', 'D67', 'D69', 'D05_SAEB'
    ],
    'Tratamento da Informação': [
        'D09_SAEB', 'D75', 'D77', 'D37_SAEB'
    ]
}

# Converte em DataFrame para visualização
df_eixos_descritores = pd.DataFrame([
    {'EIXO': eixo, 'DESCRITOR': descritor}
    for eixo, descritores in eixos_classificados.items()
    for descritor in descritores
])

In [17]:
# Adicionar coluna de eixo ao comparativo completo
comparativo_completo_com_eixo = comparativo_completo.copy()
comparativo_completo_com_eixo["Eixo"] = comparativo_completo_com_eixo.index.map(
    lambda d: next((eixo for eixo, descritores in eixos_classificados.items() if d in descritores), None)
)

# Filtrar apenas descritores com eixo identificado
comparativo_completo_com_eixo = comparativo_completo_com_eixo.dropna(subset=["Eixo"])

# Calcular média por eixo para TFP e Top 10 Geral
medias_por_eixo = comparativo_completo_com_eixo.groupby("Eixo")[
    ["TFP - (%)", "Top 10 Geral - Média (%)"]
].mean().round(1)

#tools.display_dataframe_to_user(name="Desempenho por Eixo - TFP x Top 10", dataframe=medias_por_eixo)

medias_por_eixo

,TFP - (%),Top 10 Geral - Média (%)
Eixo,,
Espaço e Forma,38.4,99.2
Grandezas e Medidas,45.1,98.4
Números e Operações,35.2,98.0
Tratamento da Informação,44.4,99.5


In [18]:
# Criar roteiro de intervenção pedagógica por eixo com base nas médias e lacunas da TFP
roteiro_intervencao = []

for eixo, row in medias_por_eixo.iterrows():
    tfp = row["TFP - (%)"]
    top10 = row["Top 10 Geral - Média (%)"]
    lacuna = top10 - tfp

    if lacuna > 60:
        prioridade = "🔴 Alta"
    elif lacuna > 40:
        prioridade = "🟠 Média"
    else:
        prioridade = "🟢 Baixa"

    roteiro_intervencao.append({
        "Eixo": eixo,
        "Desempenho TFP (%)": tfp,
        "Média Top 10 (%)": top10,
        "Lacuna (%)": lacuna,
        "Prioridade de Intervenção": prioridade,
        "Sugestão": f"Revisar conteúdos de {eixo.lower()}, com foco nos descritores mais críticos. Propor atividades diagnósticas, planos de aula específicos e grupos de reforço."
    })


df_roteiro_intervencao = pd.DataFrame(roteiro_intervencao)

#import ace_tools as tools; tools.display_dataframe_to_user(name="Roteiro de Intervenção Pedagógica por Eixo", dataframe=df_roteiro_intervencao)

df_roteiro_intervencao

,Eixo,Desempenho TFP (%),Média Top 10 (%),Lacuna (%),Prioridade de Intervenção,Sugestão
0,Espaço e Forma,38.4,99.2,60.8,🔴 Alta,"Revisar conteúdos de espaço e forma, com foco ..."
1,Grandezas e Medidas,45.1,98.4,53.3,🟠 Média,"Revisar conteúdos de grandezas e medidas, com ..."
2,Números e Operações,35.2,98.0,62.8,🔴 Alta,"Revisar conteúdos de números e operações, com ..."
3,Tratamento da Informação,44.4,99.5,55.1,🟠 Média,"Revisar conteúdos de tratamento da informação,..."


In [3]:
import pandas as pd
xls = pd.ExcelFile('Planilhao_TCT_SPAECE_EF_2023_MT_240402-2 (1).xlsx')
df = pd.DataFrame(xls.parse('ESCOLA'))
df.columns = df.iloc[0]; df = df[1:]
df = df.drop(['CD_REDE','CD_ETAPA', 'CD_REGIONAL','CD_MUNICIPIO', 'CD_ESCOLA'], axis=1)
df = df.dropna(axis=1, how='all')
df_9ano = df[df['DC_ETAPA'] == 'ENSINO FUNDAMENTAL DE 9 ANOS - 9º ANO'].reset_index(drop=True)

df_9ano


,DC_REDE,DC_ETAPA,NM_REGIONAL,NM_MUNICIPIO,NM_ESCOLA,D01,D01_5EF,D02,D03,D04,...,D63,D65,D66,D67,D69,D73,D74,D75,D77,VL_D
0,ESTADUAL,ENSINO FUNDAMENTAL DE 9 ANOS - 9º ANO,BATURITE,ARATUBA,ESCOLA INDIGENA MANOEL FRANCISCO DOS SANTOS,-,11.111111,-,-,-,...,-,21.428571,-,53.846154,7.692308,-,-,30.769231,38.461538,35.897436
1,ESTADUAL,ENSINO FUNDAMENTAL DE 9 ANOS - 9º ANO,CANINDE,CANINDE,ESCOLA INDIGENA EXPEDITO OLIVEIRA ROCHA,-,0,-,-,-,...,-,58.333333,-,50,55.555556,-,-,77.777778,54.545455,42.307692
2,ESTADUAL,ENSINO FUNDAMENTAL DE 9 ANOS - 9º ANO,CRATEUS,CRATEUS,ESCOLA INDIGENA RAIZES DE CRATEUS,-,50,-,-,-,...,-,27.272727,-,18.181818,80,-,-,91.666667,36.363636,49.704142
3,ESTADUAL,ENSINO FUNDAMENTAL DE 9 ANOS - 9º ANO,CRATEUS,MONSENHOR TABOSA,ESCOLA INDIGENA ABA KATU,-,60,-,-,-,...,-,14.285714,-,50,33.333333,-,-,50,85.714286,48.076923
4,ESTADUAL,ENSINO FUNDAMENTAL DE 9 ANOS - 9º ANO,CRATEUS,MONSENHOR TABOSA,ESCOLA INDIGENA DE MONSENHOR TABOSA,-,100,-,-,-,...,-,69.230769,-,100,92.857143,-,-,76.923077,100,92.820513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2078,MUNICIPAL,ENSINO FUNDAMENTAL DE 9 ANOS - 9º ANO,TIANGUA,VICOSA DO CEARA,REGINALDO CARNEIRO DA CUNHA EEF,-,20,-,-,-,...,-,46.666667,-,80,71.428571,-,-,71.428571,53.846154,56.730769
2079,MUNICIPAL,ENSINO FUNDAMENTAL DE 9 ANOS - 9º ANO,TIANGUA,VICOSA DO CEARA,SALUSTIANO DA COSTA CARDOSO EEIF,-,37.5,-,-,-,...,-,50,-,66.666667,53.846154,-,-,46.153846,30.769231,41.794872
2080,MUNICIPAL,ENSINO FUNDAMENTAL DE 9 ANOS - 9º ANO,TIANGUA,VICOSA DO CEARA,SANTA BARBARA EEF,-,41.666667,-,-,-,...,-,50,-,73.333333,37.5,-,-,50,38.888889,42.510121
2081,MUNICIPAL,ENSINO FUNDAMENTAL DE 9 ANOS - 9º ANO,TIANGUA,VICOSA DO CEARA,TRAPIA EEF,-,12.5,-,-,-,...,-,54.545455,-,57.142857,57.142857,-,-,63.157895,52.380952,50.961538


In [13]:
#Média Geral 
media_geral = df_9ano["VL_D"].mean().round(1)
display(media_geral)

# Média escolas municipais
media_municipal = df_9ano[df_9ano["DC_REDE"].str.upper() == "MUNICIPAL"]["VL_D"].mean().round(1)
display(media_municipal)

# Média escolas estaduais
media_estadual = df_9ano[df_9ano["DC_REDE"].str.upper() == "ESTADUAL"]["VL_D"].mean().round(1)
display(media_estadual)

# Média escolas de Fortaleza
media_fortaleza = df_9ano[df_9ano["NM_MUNICIPIO"].str.upper() == "FORTALEZA"]["VL_D"].mean().round(1)
display(media_fortaleza)

# Total de escolas analisadas
escolas_analisadas = df_9ano["NM_ESCOLA"].count()
display(escolas_analisadas)

# Média Escola selecionada
media_df = df_9ano[df_9ano["NM_ESCOLA"] == "ESCOLA INDIGENA MANOEL FRANCISCO DOS SANTOS"]
media_df = media_df["VL_D"].mean().round(1)
display(media_df)

# Média Escola selecionada por respectiva rede


# Ranking geral das escolas
df_ranking = df_9ano.sort_values(by="VL_D", ascending=False).reset_index(drop=True)
df_ranking['Ranking'] = df_ranking.index + 1
df_ranking = df_ranking[df_ranking["NM_ESCOLA"] == "ESCOLA INDIGENA MANOEL FRANCISCO DOS SANTOS"]
df_ranking = df_ranking.reset_index(drop=False)
df_ranking = df_ranking["Ranking"].values
# display(f"teste {df_ranking}")
display(df_ranking)

# # Ranking escola selecionada por respectiva rede
# df_rede = df_9ano.sort_values(by="VL_D", ascending=False).reset_index(drop=True)
# df_rede = df_rede[df_rede["DC_REDE"].str.upper() == "ESTADUAL"].reset_index(drop=True)
# df_rede['Ranking'] = df_rede.index + 1
# df_rede = df_rede[df_rede["NM_ESCOLA"] == "ESCOLA INDIGENA MANOEL FRANCISCO DOS SANTOS"]
# display(df_rede["Ranking"])

# # Rede da escola selecionada
# rede_escola = df_9ano[df_9ano["NM_ESCOLA"] == "ESCOLA INDIGENA MANOEL FRANCISCO DOS SANTOS"]["DC_REDE"].values[0]
# display(rede_escola)


np.float64(53.3)

np.float64(53.7)

np.float64(40.3)

np.float64(40.4)

np.int64(2083)

np.float64(35.9)

array([1869])

In [24]:
df_9ano_estadual = df_9ano[df_9ano["DC_REDE"].str.upper() == "ESTADUAL"]